In [173]:
from bs4 import BeautifulSoup

In [174]:
import StringIO
import argparse

In [175]:
# Argparse declerations
parser = argparse.ArgumentParser()
# Argparse Arguments
parser.add_argument('books_list', nargs='+')
parser.add_argument('output_file', nargs='+')
parser.add_argument('toc_name')
parser.add_argument('ncx_name', nargs='+' )
parser.add_argument('--numbered_toc', dest='feature', action='store_false')
parser.add_argument('--list_toc', dest='feature', action='store_true')
parser.set_defaults(feature=True)





In [176]:
#variable declerations
# counter for play order
counter = 3 
# main loop counter
mainloop = True


In [177]:
#args = parser.parse_args('scanner_input.html A_Scanner_Darkly.html ScannerToc.html ScannerNcx.ncx'.split())
#print(args)
args = parser.parse_args()
# I know this is sloppy, but it works to switch between numbered or non numbered TOCs
feature = args.feature
print(args)

Namespace(books_list=['scanner_input.html'], feature=True, ncx_name=['ScannerNcx.ncx'], output_file=['A_Scanner_Darkly.html'], toc_name='ScannerToc.html')


In [178]:
# Templates 
NUMTOCTEMPLATE = BeautifulSoup("<html><head><title>Table Of Contents</title><link rel='stylesheet' type='text/css' href='c:\users\\netmier\documents\\books in progress\python scripts\genStyleSheet.css'></head><body><ol></ol></body</html>")
LISTTOCTEMPLATE = BeautifulSoup("<html><head><title>Table Of Contents</title><link rel='stylesheet' type='text/css' href='c:\users\\netmier\documents\\books in progress\python scripts\genStyleSheet.css'></head><body><ul></ul></body</html>")

NCXTEMPLATE = BeautifulSoup("""
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE ncx PUBLIC '-//NISO/DTD ncx 2005-1/EN' 'http://www.daisy.org/z3986/2005/ncx-2005-1.dtd'>
<ncx xmlsn='http://www.daisy.org z3986/2005/ncx/' version='2005-1' xml:lang='en-US'>
<head>
<meta content='uid' name='dtb:uid'/>
<meta content='1' name='dtb:depth'/>
<meta content='0' name='dtb:totalPageCount'/>
<meta content='0' name='dtb:maxPageNumber'/>
</head>
<docTitle><text></text></docTitle>
<docAuthor><text></text></docAuthor>
<navMap>
</navMap>
</ncx>""", "xml")

In [179]:
def harvest(books_list, output_file):
    '''used to gather several source files into one large file for processing'''
    #outbook = output_file[0]
    with open(output_file[0], "w+") as outfile:
        for bname in books_list:
            with open(bname, "r") as readfile:
                outfile.write(readfile.read() + "\n\n")


In [180]:
def tocnumbered(soup):
    '''generates a numbered,linked table of contents page'''
    global NUMTOCTEMPLATE
    finalToc = open(args.toc_name, "w+")
    for tag in reversed(soup.find_all("a", attrs={"id":True})):
        NUMTOCTEMPLATE.body.ol.insert(0,NUMTOCTEMPLATE.new_tag("a"))
        NUMTOCTEMPLATE.a["href"] = args.output_file[0] + '#' + (tag["id"])
        NUMTOCTEMPLATE.a.string = tag.h2.string
    for tag in NUMTOCTEMPLATE.find_all("a"): #this wraps the tags with <li> tags.
        tag.wrap(NUMTOCTEMPLATE.new_tag("li")) 
    print >> finalToc, NUMTOCTEMPLATE.prettify()
    finalToc.close()

In [181]:
def toclist(soup):
    '''generates a linked table of contents page'''
    global LISTTOCTEMPLATE
    finalToc = open(args.toc_name, "w+")
    for tag in reversed(soup.find_all("a", attrs={"id":True})):
        LISTTOCTEMPLATE.body.ul.insert(0,LISTTOCTEMPLATE.new_tag("a"))
        LISTTOCTEMPLATE.a["href"] = args.output_file[0] + '#' + (tag["id"])
        LISTTOCTEMPLATE.a.string = tag.h2.string
    for tag in LISTTOCTEMPLATE.find_all("a"): #this wraps the tags with <li> tags.
        tag.wrap(LISTTOCTEMPLATE.new_tag("li")) 
    print >> finalToc, LISTTOCTEMPLATE.prettify()
    finalToc.close()

In [182]:
def ncxmaker(soup):
    '''generates an ncx file for use in ebook creation'''
    global counter
    global NCXTEMPLATE
    for tag in reversed(soup.find_all("a", attrs={"id":True})):
        NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = tag["id"], playorder = 0))
        NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = args.output_file[0] + '#' + (tag["id"])))
        NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
        NCXTEMPLATE.navLabel.string = tag.h2.string
        NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))
    for tag in NCXTEMPLATE.find_all("navpoint"): 
        tag["playorder"] = counter
        counter = counter + 1
    NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = "id", playorder = 2))
    NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = "" ))
    NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
    NCXTEMPLATE.navLabel.string = ""
    NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))
    NCXTEMPLATE.navMap.insert(0, NCXTEMPLATE.new_tag("navpoint", id = "id", playorder = 1))
    NCXTEMPLATE.navMap.navpoint.insert(2,NCXTEMPLATE.new_tag("content", src = "" ))
    NCXTEMPLATE.navMap.navpoint.insert(0, NCXTEMPLATE.new_tag("navLabel"))
    NCXTEMPLATE.navLabel.string = ""
    NCXTEMPLATE.navLabel.string.wrap(NCXTEMPLATE.new_tag("text"))

In [183]:
def makefiles():
    '''simply takes the templates that were filled out and prints them to files'''
    #finalToc = open(args.toc_name[0], "w+")
    finalNcx = open(args.ncx_name[0], "w+")
    #print >> finalToc, TOCTEMPLATE.prettify()
    print >> finalNcx, NCXTEMPLATE.prettify()
    #finalToc.close()
    finalNcx.close()

In [184]:
harvest(args.books_list,args.output_file)

In [185]:
print(args.output_file[0])

A_Scanner_Darkly.html


In [186]:
the_book = open(args.output_file[0])

In [187]:
soup = BeautifulSoup(the_book)


In [188]:
if feature is True:
    toclist(soup)
if feature is False:
    tocnumbered(soup)

In [189]:
ncxmaker(soup)


In [190]:
makefiles()